# BAD POEMS by Edgar Allan Poe - Computer Version #

## 1. Inspirational set ##
We firstly chose and created an inspirational poem set. We have delineated our anthology to exclusively feature the poetic works of Edgar Allan Poe, with a particular focus on those that explore themes of love, romance, mystery, and the interplay between these concepts.  It is noteworthy that rather than including entire poems, we have thoughtfully excerpted select verses from various Poe compositions. This selective extraction serves to enhance the diversity and inspirational resonance of our curated poem set.

This set was saved in a .txt file, which will be further preprocessed. As such, the words were written in lowercase and any punctuation was removed.


In [2]:
import random
import os
import string
import nltk
import pronouncing
from nltk.corpus import cmudict

pronouncing_dict = cmudict.dict()

# Load and preprocess input poems sentences
with open('poem_inspiration.txt', 'r') as file:
    poems_inspiration = file.read().split('\n')

# Function to preprocess the poem_inspiration by removing punctuation and converting to lowercase
def preprocess_poem(poems):
    clean_inspiration = poems.translate(str.maketrans('', '', string.punctuation)).lower()
    return clean_inspiration

poems_inspiration = [preprocess_poem(poem) for poem in poems_inspiration]

## 2. Model Approach ##
We decided on the Markov chain approach. In the context of natural language processing and text generation, this approach is used to model the probability of transitioning from one word or phrase to another, allowing them to generate text that appears coherent and resembles the patterns observed in the training data. 

In our context, the Markov chain analyzes a text corpus and generates new text by selecting the next word based on the probability distribution of words that follow the current word. 


In [3]:
# Build a Markov chain model based on inspiring poems
def build_markov_chain(poems):
    chain = {}
    for poem in poems:
        words = poem.split()
        for i in range(len(words) - 1):
            current_word = words[i]
            next_word = words[i + 1]
            if current_word in chain:
                chain[current_word].append(next_word)
            else:
                chain[current_word] = [next_word]
    return chain

markov_chain = build_markov_chain(poems_inspiration)


## 3. Rhyme Creation ##
Additionally, we thought about creating rhymes for the obtained poems. The rhymes are done between two consecutive lines: if the last word from the second line doesn't rhyme with the last word from the first line, it will be changed (the word from the second line) with another word from the dictionary that rhymes. 

Selecting how the rhymes should be accomplished was a laborious task, and we concluded by combing two approaches: 
1. creating rhymes based on the last 3 characters of the word for those whose length is greater than 2
2. creating rhymes based on the pronunciation for the smaller words (of one or two letters)

The second method was implemented to counter the problem when lines end with 'I'. (Edgar Allan Poe adeptly employed the inversion technique within his poetic compositions, a literary device characterized by the interchange of pronouns and verbs. Consequently, drawing inspiration from such an artful practice, our automated poem generator occasionally incorporates similar structural inversion into its generated verses.)

Moreover, the pronunciation approach exhibits a certain shortcoming, with regard to the occasional inability to identify rhyming words contained within the English lexicon. Thus, further preprocessing measures were implemented to rectify this inconvenience, involving the removal of rhyming words that do not align with established English vocabulary.

For diversity purposes, we chose not to use the same rhyme twice in a generated poem. 
As a result, whenever a rhyme is introduced, we check its prior usage among the various potential rhymes. If it has not been previously employed, the current rhyme is integrated into the verse, conversely, if it has already been utilized, we choose the next rhyme from the pool of available rhymes.

Finally, we thought "Shouldn't grammar matter when selecting the rhymes?". The prevailing answer was "YES, for example we cannot have two verbs next to each other". To tackle this issue we used the pos tags delivered by the NLTK package in the following way: 
   
If the second last word was: 
1. __noun__ or __personal pronoun ->__ the rhyme should be a __verb__
2. __verb ->__ the rhyme should be an __adjective__
3. __adjective__, __possessive pronoun__, __determiner__ or __preposition ->__ the rhyme should be a __noun__


In [ ]:
def find_rhymes(word):
    # Get the phonetic representation of the input word
    word = word.lower()
    rhyme_with = word[-3:]

    # Find words with the same phonetic ending
    rhymes = []
    for w, phonemes in pronouncing_dict.items():
        rhyme = w[-3:]
        if rhyme == rhyme_with:
            rhymes.append(w)
    return rhymes


def find_rhymes_pronounced(word):
    rhymes = pronouncing.rhymes(word)

    correct_rhymes = [rhyme for rhyme in rhymes if rhyme in pronouncing_dict.values()]
    return correct_rhymes


def get_most_common_pos_tag(word):
    tokens = nltk.word_tokenize(word)
    pos_tags = nltk.pos_tag(tokens)
    for elem in pos_tags:
        tag = elem[1]
        return tag


def choose_rhyme(rhymes, tag, index):
    if tag[:2] == 'NN':
        after_tag = 'VB'
    elif tag[:2] == 'VB':
        after_tag = 'JJ'
    elif tag[:2] == 'JJ':
        after_tag = 'NN'
    elif tag == 'PRP':
        after_tag = 'VB'
    elif tag == 'PRP$':
        after_tag = 'NN'
    elif tag == 'IN':
        after_tag = 'NN'
    elif tag == 'DT':
        after_tag = 'NN'
    else:
        after_tag = 'NN'
    for rhyme in rhymes:
        if get_most_common_pos_tag(rhyme) == after_tag:
            return rhyme
    if len(rhymes) > 1:
        return rhymes[index]
    else:
        return None


## 4. Poem Generator ##
In this part of the code, we consolidate all the gathered information and created functions to implement the poem generator. 

In [6]:
def generate_poem(chain, words_per_line, lines=4):
    poem = []
    # Array to store the rhymes for further checks
    used_rhymes = []
    index = 0

    for _ in range(lines):
        current_word = random.choice(list(chain.keys()))
        line = [current_word]

        for _ in range(words_per_line - 1):
            if current_word in chain:
                next_word = random.choice(chain[current_word])
                if next_word == 'i': next_word = 'I'
                line.append(next_word)
                current_word = next_word
            else:
                break

        if line:
            line[0] = line[0].capitalize()

        # A rhyme needs to be added only for even lines
        if len(poem) % 2 != 0:
            first_line = poem[len(poem) - 1]
            second_line = line
            lista = first_line.split(' ')
            last_word = lista[len(lista) - 1]
            if len(last_word) >= 3:
                rhymes = find_rhymes(last_word)
            else:
                rhymes = find_rhymes_pronounced(last_word)


            # Get second-last word of second line to set a speech tag for rhyme
            second_last = second_line[len(second_line) - 2]
            tag_second_last = get_most_common_pos_tag(second_last)
            rhyme = choose_rhyme(rhymes, tag_second_last, index)
            if rhyme in used_rhymes:
                index = index+1
            if rhyme is not None:
                line.append(rhyme)

        poem.append(' '.join(line))

    return '\n'.join(poem)


## 5. Book of poems ##
Like any reputable and accomplished poet, our poem generator doesn't merely create a single poem but an entire anthology. The number of poems in the anthology is determined by the user's choice and preference.

For our examples we selected the poems to be written on 8 lines and the verses to have a maximum of 4 words. 

Another noteworthy aspect is that for the poems' presentation, famous chatGPT was asked, given the poem as input, to choose a compelling and memorable title for it. 

In [9]:
def create_poems_book(poems_directory, num_poems):
    if not os.path.exists(poems_directory):
        os.makedirs(poems_directory)

    for i in range(num_poems):
        poem_content = generate_poem(markov_chain, 4, 8)
        print(poem_content)
        print('\n')
        poem_filename = os.path.join(poems_directory, f'poem{i + 1}.txt')
        with open(poem_filename, 'w') as f:
            f.write(poem_content)


create_poems_book('poems_book', 3)
print("poems book successfully generated")

This kiss upon the
Grey eye glances absinthe
Nymph from fairyland
Green isle in what ackland
Them with a talisman
Its soullife ackerman
God can I
Languid skies


Gold can never buy
Under the very heaven overbuy
Take this kiss upon
Hear thy grey eye capon
Light of harps unknown
Lie dead on high abbotstown
Never buy
So shake the fever overbuy


What eternal condor years
Never buy aerostars
Single kiss upon her
Is not that dear absher
Take this kiss upon
Truth that my heart capon
Mute motionless aghast
Is not that years abreast


poems book successfully generated


## 6. Evaluation ##

### 6.1 Evaluation on Edgar Allan Poe poems ###
The evaluation began with an expansion of the inspirational set derived from Edgar Allan Poe's work to encompass 229 verses. As a result, the outcomes included a diverse collection of poems that seamlessly blended distinct genres.

In [11]:
with open('EdgarAllanPope_poem_inspiration.txt', 'r') as file:
    poems_inspiration = file.read().split('\n')  # Split poems by newline


# Function to preprocess the poem_inspiration by removing punctuation and converting to lowercase
def preprocess_poem(poems):
    clean_inspiration = poems.translate(str.maketrans('', '', string.punctuation)).lower()
    return clean_inspiration


poems_inspiration = [preprocess_poem(poem) for poem in poems_inspiration]


def create_poems_book(poems_directory, num_poems):
    if not os.path.exists(poems_directory):
        os.makedirs(poems_directory)

    for i in range(num_poems):
        poem_content = generate_poem(build_markov_chain(poems_inspiration), 4, 8)
        print(poem_content)
        print('\n')
        poem_filename = os.path.join(poems_directory, f'poem{i + 1}.txt')
        with open(poem_filename, 'w') as f:
            f.write(poem_content)

create_poems_book('poems_book', 3)
print("poems book successfully generated")

Night or you here
Grave and I jere
Wandering from the stillness
Most knowing eye glances abruptness
Thing of the future
Engaged in a stately acupuncture
Dissever my life and
Silence was many a ackland

Head at heart search
Songs one word lenore arch
Nor the silken sad
Burden bore her up assad
Beast upon a most
Alphabet to dream before accost
Fiend I
Green isle in a

Each separate dying ember
Feel the nighttide I aber
Thee by me dreams
Grew stronger hesitating then abrahams
Lonely on the lost
Turning all that didst accost
Forgiveness I
Flitting still a demons


poems book successfully generated


### 6.2 Evaluation on verses from songs ###
In this scenario, we aimed to assess our generator's performance when utilizing an inspirational dataset comprising verses from Eminem's rap songs. This dataset consists of 100 verses.

In [13]:
with open('Eminem_inspirational set.txt', 'r') as file:
    poems_inspiration = file.read().split('\n')  # Split poems by newline


# Function to preprocess the poem_inspiration by removing punctuation and converting to lowercase
def preprocess_poem(poems):
    clean_inspiration = poems.translate(str.maketrans('', '', string.punctuation)).lower()
    return clean_inspiration


poems_inspiration = [preprocess_poem(poem) for poem in poems_inspiration]


def create_poems_book(poems_directory, num_poems):
    if not os.path.exists(poems_directory):
        os.makedirs(poems_directory)

    for i in range(num_poems):
        poem_content = generate_poem(build_markov_chain(poems_inspiration), 6, 8)
        print(poem_content)
        print('\n')
        poem_filename = os.path.join(poems_directory, f'poem{i + 1}.txt')
        with open(poem_filename, 'w') as f:
            f.write(poem_content)

create_poems_book('Eminem\'s_poems_book', 3)
print("poems book successfully generated")

In the bar toasting to me
Toasting to meet a freakish little
Around to me dad I
Truly the sound of holding you
Second chance to unpack his hands
Much passion youd expect this chance abounds
Livelihood I
Her yet


Weird again hes acting weird again
Not a holiday abstain
On his back like that its
Last look at night I acquits
Walk in my voice
Keep making that face till it accomplice
Loved someone so at my voice
Dream of sin doer this one accomplice

Down and its a dramatic rate
That every time you do this corporate
Dramatic rate
Let em say that white tank abate
Kinda feels like all eyes on
Whats her face till it feels
Sound of holding you do to
Take his own mother


poems book successfully generated


### 6.3 Evaluation Results ###
Expanding the number of verses within the Edgar Allan Poe inspirational set led to the generation of more enriched and diverse poems.

This improvement can be attributed to several factors:

1. A larger dataset allows the model to capture a wider range of themes, moods, and stylistic elements found in Poe's work. Consequently, the generated poems exhibit greater diversity, reflecting the nuances of Poe's literary legacy.

2. With more verses, the model gains a deeper understanding of the intricate nuances within Poe's poetry. This heightened comprehension enables it to produce poems that more faithfully reflect the essence of Poe's unique writing style.


For the Eminem inspirational set, an additional preprocessing step is required. This step involves a meticulous treatment of the inspirational set since expressions are often shortened using apostrophes (e.g., 'does not' is written as 'doesn't,' and 'I am' is written as 'I'm'). The current preprocessing function does not address this issue. However, tackling this preprocessing concern is essential to enhance the performance of the Markov chain and to generate better poems. As such, we implemented the following code to combat this issue, which can be easily applied to our previous inspirational set. However, it is important to highlight that in Edgar Allan Poe's poems, these shortened expressions are not present.

Comparing the Eminem inspirational set with the Edgar Allan Poe inspirational set as the sources used to generate poems, we observe a marked contrast. The Markov model, which relies heavily on probabilities, plays a pivotal role in shaping the outcomes. However, the choice of source material is paramount. In the case of the Eminem set, which includes more contemporary vocabulary, the poems tend to lose a degree of their overall expressionism. This is due to the inherent challenge of probabilistically generating verses that capture the depth and coherence found in a more established literary body, such as Edgar Allan Poe's poetry.

In [15]:
import re
with open('Eminem_inspirational set.txt', 'r') as file:
    poems_inspiration = file.read().split('\n')  # Split poems by newline


# Function to preprocess the poem_inspiration by removing punctuation and converting to lowercase
def preprocess_poem(poem):
    # Convert to lowercase
    poem = poem.lower()
    # Replace contractions 
    poem = re.sub(r"i'm", "i am", poem)
    poem = re.sub(r"he's", "he is", poem)
    poem = re.sub(r"she's", "she is", poem)
    poem = re.sub(r"it's", "it is", poem)
    poem = re.sub(r"that's", "that is", poem)
    poem = re.sub(r"there's", "there is", poem)
    poem = re.sub(r"what's", "what is", poem)
    poem = re.sub(r"where's", "where is", poem)
    poem = re.sub(r"how's", "how is", poem)
    poem = re.sub(r"\'ll", " will", poem)
    poem = re.sub(r"\'re", " are", poem)
    poem = re.sub(r"\'ve", " have", poem)
    poem = re.sub(r"\'d", " would", poem)
    poem = re.sub(r"\'t", " not", poem)
    
    # Remove all the punctuation
    poem = re.sub(r"[\,.?:;_'!()\"-]", "", poem)
    return poem


poems_inspiration = [preprocess_poem(poem) for poem in poems_inspiration]


def create_poems_book(poems_directory, num_poems):
    if not os.path.exists(poems_directory):
        os.makedirs(poems_directory)

    for i in range(num_poems):
        poem_content = generate_poem(build_markov_chain(poems_inspiration), 6, 8)
        print(poem_content)
        print('\n')
        poem_filename = os.path.join(poems_directory, f'poem{i + 1}.txt')
        with open(poem_filename, 'w') as f:
            f.write(poem_content)

create_poems_book('Eminem\'s_poems_book', 3)
print("poems book successfully generated")

In the next she is her
Check the clock absher
Writing a chess match she is
Empty hand
With what day
Flow and right now there is aday
Gonna write itself
Throwing roses at my reason for bookshelf

Kinda feels like a jealous man
Try to love you smiling ackerman
Temperature in the curtain closes they
Passion you know they are truly achey
Rest of you would fall apart
Wait but she is a flow ahart
Friday and I
Ever since I

The blame
Wrong with dad I adame
Write itself
Your face till it is me bookshelf
Really beggining to love you and
Holding you know that bought me ackland
Him in the temperature in the
Steel knife absinthe


poems book successfully generated


### 7. Presenting poems book ###
To adequately present the remarkable showcase of our Bad Poet program, we harnessed the capabilities of advanced Generative AI systems. The presentation process involved the utilization of three distinct cloud applications, each playing a vital role in shaping the final outcome.

1. Stable Diffusion - the generative AI model, known for its capacity to produce high-quality content by employing a diffusion process. To accurately generate results, the concept of a "prompt" was leveraged. We meticulously defined the attributes and provided detailed neat description of the desired output. Through this process, we expertly crafted the title page of our poetry collection.
2.  Dream Studio - Dream Studio's model, "SDXL v1.0" emerged as a key factor in shaping the layout and imagery for the subsequent three pages, each dedicated to presenting our collection of poems. By formulating concise yet instructive prompts, we successfully generated three thematically aligned illustrations, artfully conveying the essence of traditional paper-bound books.
    ```
    minimal book illustration, (book description)
    ```
3. Canva - in the final phase, the materials generated thus far were meticulously curated and assembled within the confines of the Canva graphic design platform. Here, our team wielded complete control over the design and format of the book with use of user-friendly and accessible way provided by Canva creator's. The result is a visually appealing and harmoniously designed publication, thoughtfully tailored to complement the thematic and rhythmic qualities of our AI-generated poetry.

The poetry collection, aptly titled
"The Ravenous Rhymes: An Ode To Awfully Poe-Tic Poetry,"
is stored in the root folder of the project, serving as a testament to the creative possibilities enabled by advanced AI systems.